# GMM Gym

Minimal working example of a GMM!

## 1. Setup

+ 1.1 Set up real deep parameters ($\lambda_0 = f(X, \theta)$)
+ 1.2 Get ValueF and optimal policy function
+ 1.3 Simulate demand and optimal pricing for N firms

## 2. Estimating $\theta$

Try $\theta$ until error < $\varepsilon$

+ 2.1. $\theta$ values generate a $\lambda_0$
+ 2.2. $\lambda_0$ + demand observations + optimal pricing generate pricing decisions
+ 2.3. $error = \left|pricing decisions - observed pricing\right|$


## 1. Setup

In [13]:
import numpy as np
from typing import Callable
%matplotlib inline
import matplotlib.pyplot as plt
import dill
import numpy as np
import pandas as pd
    
import sys
sys.path.append('../')
import src

# Simulation parameters
########################
σerror=0.01
Nfirms = 300
time_periods = 40

def lambda_0(x) -> np.ndarray:
    """
    Generate a vector of lambdas on the observables x
    """
    lambda1 = np.clip(0.1 + 2*x + np.random.normal(0, σerror), 0, 1)
    lambda2 = np.clip(0.3 + -1*x + np.random.normal(0, σerror), 0, 1)
    lambda3 = 1 - lambda1 - lambda2
    return np.array([lambda1, lambda2, lambda3])

def get_lambdas_from_x(xs):
    y = np.empty((len(xs), 3))
    for i in range(len(xs)):
        y[i] = lambda_0(xs[i])
    return y

xs = np.abs(np.random.normal(0, 0.18, size=Nfirms))
lambdas0 = get_lambdas_from_x(xs)
lambdas0[0:10]



array([[0.5201502 , 0.09087682, 0.38897297],
       [0.63474605, 0.03872408, 0.32652987],
       [0.39151858, 0.16821543, 0.44026599],
       [1.        , 0.        , 0.        ],
       [0.24783417, 0.22661698, 0.52554884],
       [0.15496147, 0.27566373, 0.56937481],
       [0.14450657, 0.26330705, 0.59218638],
       [0.13501405, 0.27721282, 0.58777313],
       [0.3555791 , 0.18648949, 0.4579314 ],
       [0.13737234, 0.2752099 , 0.58741776]])

In [3]:
file_n = "2018-10-5vfi_dict.dill"
with open('../data/' + file_n, 'rb') as file:
    data_d = dill.load(file)

lambdas = src.generate_simplex_3dims(n_per_dim=data_d['n_of_lambdas_per_dim'])
price_grid = np.linspace(data_d['min_price'], data_d['max_price'])

policy = data_d['policy']
valueF = data_d['valueF']
lambdas_ext = src.generate_simplex_3dims(n_per_dim=
                                         data_d['n_of_lambdas_per_dim'])

#Interpolate policy (level price). valueF is already a function
policyF = src.interpolate_wguess(lambdas_ext, policy)


# Demand realization
#src.draw_true_log_dmd(price, β, σϵ):




In [18]:
def one_run(lambda0=np.array([0.4, 0.4, 0.2]),
                             true_beta=src.betas_transition[2],
                             dmd_σϵ=src.const.σ_ɛ, maxt=time_periods):
    current_lambdas = lambda0
    d = {}
    d['level_prices'] = []
    d['log_dmd'] = []
    d['valueF'] = []
    d['lambda1'] = []
    d['lambda2'] = []
    d['lambda3'] = []


    for t in range(time_periods):
        d['lambda1'].append(current_lambdas[0])
        d['lambda2'].append(current_lambdas[1])
        d['lambda3'].append(current_lambdas[2])
        d['valueF'].append(valueF(current_lambdas[:2])[0])

        #0. Choose optimal price (last action of t-1)
        level_price = policyF(current_lambdas[:2]) #Check: Is this correctly defined with the first two elements?
        d['level_prices'].append(level_price[0])

        #1. Demand happens
        log_dmd = src.draw_true_log_dmd(level_price, true_beta, dmd_σε)
        d['log_dmd'].append(log_dmd[0])

        #2. lambda updates: log_dmd: Yes, level_price: Yes
        new_lambdas = src.update_lambdas(log_dmd, src.dmd_transition_fs, current_lambdas,
                       action=level_price, old_state=1.2)

        current_lambdas = new_lambdas
            
    return pd.DataFrame(d)

def many_runs(nfirms, **kwargs):
    dfs = []
    for firm_i in range(nfirms):
        df = one_run(lambda0=lambdas0[firm_i], **kwargs)
        df['firm'] = firm_i
        dfs.append(df)
        
    return pd.concat(dfs, axis=0)

df = many_runs(Nfirms)

## Estimating $\theta$

+ 2.1. $\theta$ values generate a $\lambda_0$


In [ ]:
def from_theta_to_lamba0(theta):
    pass

+ 2.2. $\lambda_0$ + demand observations + optimal pricing generate pricing decisions

In [ ]:
def generate_pricing_decisions(lambda0, demand_obs):
    pass

+ 2.3. $error = \left|pricing decisions - observed pricing\right|$